In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import biosppy.signals.ecg as ecg
import numpy as np
import neurokit2 as nk
from scipy.signal import butter, filtfilt

In [2]:
path = "DREAMER.mat"
raw = sio.loadmat(path)

In [3]:
p_wave_df= pd.DataFrame(columns = ['p_mean','p_std','p_minimum','p_maximum','p_wave_range'])
t_wave_df= pd.DataFrame(columns = ['t_mean','t_std','t_minimum','t_maximum','t_wave_range'])
r_wave_df= pd.DataFrame(columns = ['r_mean','r_std','r_minimum','r_maximum','r_wave_range'])

In [4]:
def statistics_caculation(waves):
    mean_w=np.nanmean(waves)
    std_w= np.nanstd(waves)
    minimum_w = np.nanmin(waves)
    maximum_w = np.nanmax(waves)
    wave_range_w = np.nanmax(waves) - np.nanmin(waves)
    return mean_w,std_w,minimum_w,maximum_w,wave_range_w 

In [5]:
def calculations( mean1,std1,minimum1,maximum1,wave_range1,mean2,std2,minimum2,maximum2,wave_range2):
    mean_b=mean1/mean2
    std_b=std1/std2
    minimum_b=minimum1/minimum2
    maximum_b=maximum1/maximum2
    wave_range_b=wave_range1/wave_range2
    return mean_b,std_b,minimum_b,maximum_b,wave_range_b 

In [6]:
def calculate_static_features(waves1,waves2,waves3,waves4):
    mean_bl,std_bl,minimum_bl,maximum_bl,wave_range_bl=statistics_caculation(waves1)
    mean_sl,std_sl,minimum_sl,maximum_sl,wave_range_sl=statistics_caculation(waves2)
    mean_br,std_br,minimum_br,maximum_br,wave_range_br=statistics_caculation(waves3)
    mean_sr,std_sr,minimum_sr,maximum_sr,wave_range_sr=statistics_caculation(waves4)
    mean_l,std_l,minimum_l,maximum_l,wave_range_l=calculations(mean_bl,std_bl,minimum_bl,maximum_bl,wave_range_bl, mean_sl,std_sl,minimum_sl,maximum_sl,wave_range_sl)
    mean_r,std_r,minimum_r,maximum_r,wave_range_r=calculations( mean_br,std_br,minimum_br,maximum_br,wave_range_br ,mean_sr,std_sr,minimum_sr,maximum_sr,wave_range_sr)
    mean=(mean_l+mean_r)/2
    std=(std_l+std_r)/2
    minimum=(minimum_l+minimum_r)/2
    maximum=(maximum_l+maximum_r)/2
    wave_range =(wave_range_l+wave_range_r)/2
    return mean,std,minimum,maximum,wave_range 
    

In [7]:
def get_beaks(raw):
    # load raw baseline and stimuli data for left and right
      # load raw baseline and stimuli data for left and right
  
    return  peaks_bl,peaks_sl,peaks_br, peaks_sr,basl_left,stim_left, basl_right,stim_right

In [8]:

 def time_domain(raw):
    l=0
    time_ECG={}
    for participant in range(0, 23):
        for video in range(0, 18):
            # load raw baseline and stimuli data for left and right
            basl_left = (raw["DREAMER"][0, 0]["Data"]
                 [0, participant]["ECG"][0, 0]
                         ["baseline"][0, 0][video, 0][:, 0])
            stim_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["stimuli"][0, 0][video, 0][:, 0])
            basl_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["baseline"][0, 0][video, 0][:, 1])
            stim_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["stimuli"][0, 0][video, 0][:, 1])
            peaks_bl, info_bl = nk.ecg_peaks( basl_left, sampling_rate=256)
            peaks_sl, info_sl = nk.ecg_peaks( stim_left, sampling_rate=256)
            peaks_br, info_br = nk.ecg_peaks( basl_right, sampling_rate=256)
            peaks_sr, info_sr = nk.ecg_peaks( stim_right, sampling_rate=256)
            psignals_basl_l, waves_basl_l =nk.ecg_delineate(basl_left,info_bl,sampling_rate=256)
            psignals_stim_l, waves_stim_l =nk.ecg_delineate(stim_left,info_sl, sampling_rate=256)
            psignals_basal_r, waves_basl_r = nk.ecg_delineate(basl_right,info_br, sampling_rate=256)
            psignals_stim_r, waves_stim_r = nk.ecg_delineate(stim_right,info_sr, sampling_rate=256)   
    
            p_mean,p_std,p_minimum,p_maximum,p_wave_range=calculate_static_features(np.array(waves_basl_l['ECG_P_Peaks']),np.array(waves_stim_l['ECG_P_Peaks']),np.array(waves_basl_r['ECG_P_Peaks']),np.array(waves_stim_r['ECG_P_Peaks']))
            r_mean,r_std,r_minimum,r_maximum,r_wave_range=calculate_static_features(np.array(info_bl['ECG_R_Peaks']),np.array(info_sl['ECG_R_Peaks']),np.array(info_br['ECG_R_Peaks']),np.array(info_sr['ECG_R_Peaks']))
            t_mean,t_std,t_minimum,t_maximum,t_wave_range=calculate_static_features(np.array(waves_basl_l['ECG_T_Peaks']),np.array(waves_stim_l['ECG_T_Peaks']),np.array(waves_basl_r['ECG_T_Peaks']),np.array(waves_stim_r['ECG_T_Peaks']))         
            P_list= list([ p_mean,p_std,p_minimum,p_maximum,p_wave_range])
            p_wave_df.loc[l] = P_list
            r_list= list([ r_mean,r_std,r_minimum,r_maximum,r_wave_range])
            r_wave_df.loc[l] = r_list
            t_list= list([t_mean,t_std,t_minimum,t_maximum,t_wave_range])
            t_wave_df.loc[l] = t_list
            l+=1
    time_ECG = pd.concat([p_wave_df,r_wave_df,t_wave_df],axis=1)
    return time_ECG

In [9]:
domain=time_domain(raw)
domain

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User

,p_mean,p_std,p_minimum,p_maximum,p_wave_range,r_mean,r_std,r_minimum,r_maximum,r_wave_range,t_mean,t_std,t_minimum,t_maximum,t_wave_range
0,0.297957,0.302031,0.934685,0.303361,0.300593,0.299000,0.302032,0.942418,0.303868,0.300583,0.300548,0.301568,0.889549,0.304967,0.301179
1,0.467389,0.450559,3.075634,0.462588,0.458902,0.468511,0.450570,2.103594,0.463157,0.458872,0.469265,0.451041,2.028810,0.463792,0.457957
2,0.176460,0.175866,0.899686,0.172699,0.171435,0.177194,0.175853,0.915396,0.173054,0.171425,0.178162,0.175870,0.821862,0.173535,0.171590
3,0.368459,0.361792,4.010638,0.366418,0.362371,0.369621,0.361761,2.670819,0.367103,0.362476,0.368277,0.358729,2.124615,0.364580,0.359214
4,0.452919,0.455058,1.103977,0.446740,0.443627,0.454089,0.455074,1.081880,0.447230,0.443526,0.454329,0.455849,1.095714,0.448406,0.443794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.359494,0.361053,0.348830,0.356227,0.356253,0.360637,0.361052,0.494681,0.356780,0.356180,0.364261,0.362877,0.621959,0.359561,0.358061
410,0.205294,0.201432,0.744318,0.197340,0.196114,0.206078,0.201412,0.786047,0.197722,0.196112,0.204951,0.199192,0.824052,0.196205,0.193974
411,0.321380,0.314175,2.462687,0.312048,0.309153,0.322389,0.314195,1.896226,0.312539,0.309166,0.319663,0.310041,1.567210,0.309164,0.304928
412,0.230104,0.238167,1.982906,0.236258,0.233132,0.227557,0.241273,0.575472,0.236728,0.235904,0.226150,0.238349,0.682468,0.234146,0.232618


In [10]:
domain

,p_mean,p_std,p_minimum,p_maximum,p_wave_range,r_mean,r_std,r_minimum,r_maximum,r_wave_range,t_mean,t_std,t_minimum,t_maximum,t_wave_range
0,0.297957,0.302031,0.934685,0.303361,0.300593,0.299000,0.302032,0.942418,0.303868,0.300583,0.300548,0.301568,0.889549,0.304967,0.301179
1,0.467389,0.450559,3.075634,0.462588,0.458902,0.468511,0.450570,2.103594,0.463157,0.458872,0.469265,0.451041,2.028810,0.463792,0.457957
2,0.176460,0.175866,0.899686,0.172699,0.171435,0.177194,0.175853,0.915396,0.173054,0.171425,0.178162,0.175870,0.821862,0.173535,0.171590
3,0.368459,0.361792,4.010638,0.366418,0.362371,0.369621,0.361761,2.670819,0.367103,0.362476,0.368277,0.358729,2.124615,0.364580,0.359214
4,0.452919,0.455058,1.103977,0.446740,0.443627,0.454089,0.455074,1.081880,0.447230,0.443526,0.454329,0.455849,1.095714,0.448406,0.443794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.359494,0.361053,0.348830,0.356227,0.356253,0.360637,0.361052,0.494681,0.356780,0.356180,0.364261,0.362877,0.621959,0.359561,0.358061
410,0.205294,0.201432,0.744318,0.197340,0.196114,0.206078,0.201412,0.786047,0.197722,0.196112,0.204951,0.199192,0.824052,0.196205,0.193974
411,0.321380,0.314175,2.462687,0.312048,0.309153,0.322389,0.314195,1.896226,0.312539,0.309166,0.319663,0.310041,1.567210,0.309164,0.304928
412,0.230104,0.238167,1.982906,0.236258,0.233132,0.227557,0.241273,0.575472,0.236728,0.235904,0.226150,0.238349,0.682468,0.234146,0.232618


In [11]:

def hrv_featues(raw):
    time_ECG={}
    frequency_ECG = {}
    non_linear={}
    ECG={}
    for participant in range(0, 23):
        for video in range(0, 18):
            sampling_rate=256
            # load raw baseline and stimuli data for left and right
            basl_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["baseline"][0, 0][video, 0][:, 0])
            stim_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["stimuli"][0, 0][video, 0][:, 0])
            basl_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["baseline"][0, 0][video, 0][:, 1])
            stim_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["stimuli"][0, 0][video, 0][:, 1])
            peaks_bl, info_bl = nk.ecg_peaks( basl_left, sampling_rate=256)
            peaks_sl, info_sl = nk.ecg_peaks( stim_left, sampling_rate=256)
            peaks_br, info_br = nk.ecg_peaks( basl_right, sampling_rate=256)
            peaks_sr, info_sr = nk.ecg_peaks( stim_right, sampling_rate=256)
            #computr HRV time
            hrv_time_l = nk.hrv_time(info_bl, sampling_rate=256)/nk.hrv_time(info_sl, sampling_rate=256)
            hrv_time_r = nk.hrv_time(info_br, sampling_rate=256)/nk.hrv_time(info_sr, sampling_rate=256)
            features_time = (hrv_time_l + hrv_time_r)/2
            # Compute HRV indices using method="welch"
            hrv_welch_l = nk.hrv_frequency(info_bl, sampling_rate=256, psd_method="welch")/nk.hrv_frequency(info_sl, sampling_rate=256, psd_method="welch")
            hrv_welch_r = nk.hrv_frequency(info_br, sampling_rate=256, psd_method="welch")/nk.hrv_frequency(info_sr, sampling_rate=256, psd_method="welch")
            features_frequency = (hrv_welch_l + hrv_welch_r)/2
            # Compute HRV non linear"
            hrv_non_linear_l =  nk.hrv_nonlinear(info_bl, sampling_rate=256)/ nk.hrv_nonlinear(info_sl, sampling_rate=256)
            hrv_non_linear_r =  nk.hrv_nonlinear(info_br, sampling_rate=256)/ nk.hrv_nonlinear(info_sr, sampling_rate=256)
            features_non_linear = (hrv_non_linear_l + hrv_non_linear_r)/2
            if not len(frequency_ECG):
                frequency_ECG = features_frequency
                time_ECG=features_time
                non_linear=features_non_linear
            else:
                frequency_ECG =pd.concat([frequency_ECG, features_frequency],ignore_index=True)
                time_ECG=pd.concat([time_ECG, features_time],ignore_index=True)
                non_linear=pd.concat([non_linear,features_non_linear],ignore_index=True)
            ECG=pd.concat([time_ECG, frequency_ECG,non_linear], axis=1)
    return ECG

In [12]:
ECG_hrv  =hrv_featues(raw)
ECG_hrv

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

...

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\User\Anaconda3\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argum

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SD1a,HRV_SD1d,HRV_SD2,HRV_SD2a,HRV_SD2d,HRV_SDNNa,HRV_SDNNd,HRV_SI,HRV_SampEn,HRV_ShanEn
0,1.019698,1.059560,NaN,NaN,NaN,NaN,NaN,NaN,1.054680,1.059518,...,1.077186,1.043695,1.060027,1.086576,1.037152,1.085724,1.037732,1.000548,1.088123,0.911529
1,0.901737,0.827695,NaN,NaN,NaN,NaN,NaN,NaN,0.638781,0.640621,...,0.643784,0.637035,0.837870,0.835036,0.840791,0.822355,0.828349,0.985735,0.731479,0.900129
2,1.033183,0.909015,NaN,NaN,NaN,NaN,NaN,NaN,0.976707,0.982355,...,1.020287,0.940447,0.904749,0.762874,1.025767,0.791541,1.018305,0.999465,1.223037,0.892552
3,0.952662,0.837676,NaN,NaN,NaN,NaN,NaN,NaN,0.795214,0.794746,...,0.716231,0.893135,0.820520,0.726417,0.883088,0.725075,0.883894,1.074804,1.141364,0.911419
4,1.063301,1.365102,NaN,NaN,NaN,NaN,NaN,NaN,1.254136,1.258122,...,1.428653,1.065992,1.302217,1.264587,1.340245,1.285440,1.309380,0.954517,0.780289,1.007655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.921319,1.159790,NaN,NaN,NaN,NaN,NaN,NaN,0.878169,0.881358,...,1.126089,0.766776,1.213732,1.399569,1.021578,1.378765,0.982155,0.887372,0.866836,0.965397
410,1.062516,1.246713,NaN,NaN,NaN,NaN,NaN,NaN,2.280957,2.294866,...,1.702592,2.881325,1.189819,1.437567,0.957814,1.450170,1.055438,1.018449,1.446031,0.858170
411,0.981267,1.300125,NaN,NaN,NaN,NaN,NaN,NaN,1.164100,1.170256,...,1.101661,1.238010,1.328836,1.344214,1.315807,1.315200,1.307720,0.983460,1.100101,0.956414
412,0.965393,1.371461,NaN,NaN,NaN,NaN,NaN,NaN,1.117095,1.119500,...,1.052785,1.210983,1.398163,1.322039,1.450797,1.264037,1.418586,1.114110,0.705521,0.931293


In [18]:
ECG_HRV_new=ECG_hrv.drop(['HRV_ULF','HRV_VLF','HRV_SDANN1','HRV_SDNNI1','HRV_SDANN2','HRV_SDNNI2','HRV_SDANN5','HRV_SDNNI5','HRV_SDNN'],axis=1)

In [19]:
ECG_HRV_new

,HRV_MeanNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_Prc20NN,...,HRV_SD1a,HRV_SD1d,HRV_SD2,HRV_SD2a,HRV_SD2d,HRV_SDNNa,HRV_SDNNd,HRV_SI,HRV_SampEn,HRV_ShanEn
0,1.019698,1.054680,1.059518,1.039092,1.034307,1.018617,1.000000,0.981730,0.850000,1.022222,...,1.077186,1.043695,1.060027,1.086576,1.037152,1.085724,1.037732,1.000548,1.088123,0.911529
1,0.901737,0.638781,0.640621,0.917890,0.708390,0.896907,0.843750,0.940733,0.787500,0.899086,...,0.643784,0.637035,0.837870,0.835036,0.840791,0.822355,0.828349,0.985735,0.731479,0.900129
2,1.033183,0.976707,0.982355,0.879820,0.945337,1.021278,0.923684,0.904412,0.906250,1.036170,...,1.020287,0.940447,0.904749,0.762874,1.025767,0.791541,1.018305,0.999465,1.223037,0.892552
3,0.952662,0.795214,0.794746,0.879300,0.834728,0.946860,0.812500,0.858099,0.791667,0.950862,...,0.716231,0.893135,0.820520,0.726417,0.883088,0.725075,0.883894,1.074804,1.141364,0.911419
4,1.063301,1.254136,1.258122,1.284039,1.179686,1.075601,1.238095,1.150950,1.259615,1.051366,...,1.428653,1.065992,1.302217,1.264587,1.340245,1.285440,1.309380,0.954517,0.780289,1.007655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.921319,0.878169,0.881358,1.258835,0.953165,0.904388,1.225000,1.354478,1.323958,0.901887,...,1.126089,0.766776,1.213732,1.399569,1.021578,1.378765,0.982155,0.887372,0.866836,0.965397
410,1.062516,2.280957,2.294866,1.173359,2.146751,1.052036,0.805556,0.765658,0.729021,1.098454,...,1.702592,2.881325,1.189819,1.437567,0.957814,1.450170,1.055438,1.018449,1.446031,0.858170
411,0.981267,1.164100,1.170256,1.324946,1.186323,0.971491,1.500000,1.544155,1.472222,0.955363,...,1.101661,1.238010,1.328836,1.344214,1.315807,1.315200,1.307720,0.983460,1.100101,0.956414
412,0.965393,1.117095,1.119500,1.420624,1.157140,0.977642,1.181818,1.208850,1.169951,0.958621,...,1.052785,1.210983,1.398163,1.322039,1.450797,1.264037,1.418586,1.114110,0.705521,0.931293


In [21]:
ECG=pd.concat([ECG_HRV_new, domain], axis=1)

In [22]:
ECG

,HRV_MeanNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_Prc20NN,...,r_mean,r_std,r_minimum,r_maximum,r_wave_range,t_mean,t_std,t_minimum,t_maximum,t_wave_range
0,1.019698,1.054680,1.059518,1.039092,1.034307,1.018617,1.000000,0.981730,0.850000,1.022222,...,0.299000,0.302032,0.942418,0.303868,0.300583,0.300548,0.301568,0.889549,0.304967,0.301179
1,0.901737,0.638781,0.640621,0.917890,0.708390,0.896907,0.843750,0.940733,0.787500,0.899086,...,0.468511,0.450570,2.103594,0.463157,0.458872,0.469265,0.451041,2.028810,0.463792,0.457957
2,1.033183,0.976707,0.982355,0.879820,0.945337,1.021278,0.923684,0.904412,0.906250,1.036170,...,0.177194,0.175853,0.915396,0.173054,0.171425,0.178162,0.175870,0.821862,0.173535,0.171590
3,0.952662,0.795214,0.794746,0.879300,0.834728,0.946860,0.812500,0.858099,0.791667,0.950862,...,0.369621,0.361761,2.670819,0.367103,0.362476,0.368277,0.358729,2.124615,0.364580,0.359214
4,1.063301,1.254136,1.258122,1.284039,1.179686,1.075601,1.238095,1.150950,1.259615,1.051366,...,0.454089,0.455074,1.081880,0.447230,0.443526,0.454329,0.455849,1.095714,0.448406,0.443794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.921319,0.878169,0.881358,1.258835,0.953165,0.904388,1.225000,1.354478,1.323958,0.901887,...,0.360637,0.361052,0.494681,0.356780,0.356180,0.364261,0.362877,0.621959,0.359561,0.358061
410,1.062516,2.280957,2.294866,1.173359,2.146751,1.052036,0.805556,0.765658,0.729021,1.098454,...,0.206078,0.201412,0.786047,0.197722,0.196112,0.204951,0.199192,0.824052,0.196205,0.193974
411,0.981267,1.164100,1.170256,1.324946,1.186323,0.971491,1.500000,1.544155,1.472222,0.955363,...,0.322389,0.314195,1.896226,0.312539,0.309166,0.319663,0.310041,1.567210,0.309164,0.304928
412,0.965393,1.117095,1.119500,1.420624,1.157140,0.977642,1.181818,1.208850,1.169951,0.958621,...,0.227557,0.241273,0.575472,0.236728,0.235904,0.226150,0.238349,0.682468,0.234146,0.232618


In [24]:
ECG.to_csv('ECG.csv')